In [8]:
import pandas as pd
import glob
# import plotly.plotly as py
import plotly.graph_objs as go

from plotly import tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) 

import numpy as np
import matplotlib.pyplot as plt
# include if using a Jupyter notebook
# %matplotlib inline

path = "/gpfs/bsc_home/xpliu/inference/vision/classification_and_detection/output-exp17/"
all_files = glob.glob(path + "*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df.head()

,benchmark,k8senv,sbatch,numctn,cscenario,cbatch,reps,qps,mean,p50,p80,p90,p95,p99,p999
0,resnet,none,32,1,offline,10,10,93.863320,130.873806,130.013855,209.791198,236.838943,250.184755,260.828805,261.780807
1,resnet,none,32,2,offline,10,10,106.083121,116.666730,116.384605,185.045075,208.467650,219.826078,228.755537,231.590579
2,resnet,none,32,4,offline,10,10,102.326792,120.201034,120.233064,191.445595,215.058790,227.102038,236.834837,239.936966
3,resnet,none,32,8,offline,10,10,100.092965,120.434395,119.770179,192.491870,216.402408,228.115927,237.360576,244.985257
4,resnet,none,32,16,offline,10,10,171.462139,71.419602,71.220294,113.861809,127.826067,135.001427,140.686948,142.203078


In [9]:
len(df)

10

In [10]:
df['benchmark'].unique()

array(['resnet'], dtype=object)

In [11]:
df['k8senv'].unique()

array(['none', 'cpumem'], dtype=object)

In [12]:
type(df['qps'][0])

numpy.float64

# Graph

In [13]:
import pandas as pd
import glob
# import plotly.plotly as py
import plotly.graph_objs as go

from plotly import tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) 

import numpy as np
import matplotlib.pyplot as plt
# include if using a Jupyter notebook
%matplotlib inline
dfg = df
dfg

,benchmark,k8senv,sbatch,numctn,cscenario,cbatch,reps,qps,mean,p50,p80,p90,p95,p99,p999
0,resnet,none,32,1,offline,10,10,93.863320,130.873806,130.013855,209.791198,236.838943,250.184755,260.828805,261.780807
1,resnet,none,32,2,offline,10,10,106.083121,116.666730,116.384605,185.045075,208.467650,219.826078,228.755537,231.590579
2,resnet,none,32,4,offline,10,10,102.326792,120.201034,120.233064,191.445595,215.058790,227.102038,236.834837,239.936966
3,resnet,none,32,8,offline,10,10,100.092965,120.434395,119.770179,192.491870,216.402408,228.115927,237.360576,244.985257
4,resnet,none,32,16,offline,10,10,171.462139,71.419602,71.220294,113.861809,127.826067,135.001427,140.686948,142.203078
5,resnet,cpumem,32,1,offline,10,10,93.547365,131.644205,130.993649,210.449652,237.109424,250.611967,261.649950,262.670326
6,resnet,cpumem,32,2,offline,10,10,108.864487,113.456429,113.654909,180.459920,202.783591,213.803347,222.929221,225.687923
7,resnet,cpumem,32,4,offline,10,10,107.256462,115.121655,115.084272,182.731062,205.503948,216.940204,226.035126,229.003085
8,resnet,cpumem,32,8,offline,10,10,104.692487,117.084301,116.776298,186.510135,209.565983,221.160863,230.296265,234.181217
9,resnet,cpumem,32,16,offline,10,10,185.539200,66.506457,66.203758,105.588535,118.803535,125.345517,130.471942,132.117246


In [16]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF
from reportlab.pdfgen import canvas

from numpy import percentile, mean, std
import plotly.graph_objects as go

from plotly.offline import init_notebook_mode, iplot

import plotly.io as pio
pio.templates.default = "none"
# pio.templates.default = "simple_white"

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
# import plotly.express as px

for x in ['qps','mean','p99']:
    for cbatch in [10]:
        for model_name in ['resnet']:
            list_traces = []
            for numctn in [1,2,4,8,16,32]: 
                dfg_bench = dfg[(dfg['benchmark']== model_name)  & (dfg['cbatch']==cbatch) & (dfg['numctn']==numctn) &\
                            ((dfg['k8senv']=='none') |\
                             (dfg['k8senv']=='cpumem')| (dfg['k8senv']=='cpumemory'))]
#                 print(dfg_bench)


                #data
                trace = go.Scatter(
                    name="ctn"+str(numctn),
                    x=[ 'none','cpumem','cpu'],
                    y= dfg_bench[x].values.tolist(),
        #             error_y = dict(type='data', array=df1_bench['model_time_std'].values.tolist()),
                    opacity=0.7,
                    mode='markers'
                )
                list_traces.append(trace)



            #layout
            layout = go.Layout(
                title={
                    'text': "exp17(cpu32)= inter32,intra16 -> inter2,intra1; restapi=32" ,
                    'y':0.85,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                width=1000,
                height=460,
                xaxis=dict(
                    title= "scenarios",
                    tickvals=[ 'none','cpumem','cpu'],
                    titlefont=dict(
                        size=10,
                        color='rgb(107, 107, 107)'
                    ),
                    tickfont=dict(
                        size=10,
                        color='rgb(107, 107, 107)'
                    )
                ),
                yaxis=dict(
                    title= x+"batch"+str(cbatch),
                    automargin=True,
                    titlefont=dict(
                        size=25,
                        color='rgb(107, 107, 107)'
                    ),
                    tickfont=dict(
                        size=25,
                        color='rgb(107, 107, 107)'
                    ),    
                ),
                legend=dict(
                    x=1.0,
                    y=1.0,
                    bgcolor='rgba(255, 255, 255, 0)',
                    bordercolor='rgba(255, 255, 255, 0)',
                    font = dict( size = 28)
                ),
                barmode='group',
                bargap=0.15,
                bargroupgap=0.08
        #         legend_orientation="h"
            )

        data = list_traces
        fig = go.Figure(data=data, layout=layout)
        fig.update_layout(legend_orientation="h")
        fig.show()    

    #     fig.write_image(path + model_name + k8senv +'.pdf')